In [1]:
from moralis import evm_api
import requests
from web3 import Web3
import time
import pandas as pd
import web3
import random
import datetime
import json
import os

In [2]:
# 获取当前路径
root = os.getcwd()

# 查看当前路径下级文件
os.listdir()

['.ipynb_checkpoints',
 'OpenOceanDA',
 'Transaction History.ipynb',
 'Code Temp',
 'Transaction History General.ipynb',
 'OpenOcean Maintainess',
 'Untitled1.ipynb',
 'Untitled.ipynb']

In [3]:
all_order_file = r'/home/jovyan/work/OpenOceanDA/LimitOrder_AllOrder.xlsx'
all_order_sheet = pd.read_excel(all_order_file)

filled_order_file = r'/home/jovyan/work/OpenOceanDA/LimitOrder_FilledOrder.xlsx'
filled_order_sheet = pd.read_excel(filled_order_file)

In [24]:
# 生成时间从而获取时间维度的数据

start_time = datetime.datetime(2021,12,30,0,0,0)
period_list = []
period_dict = {}

def period_generator(start_time):
    time_counter = start_time
    counter = 0
    
    while time_counter < datetime.datetime.now():
        time = time_counter.strftime("%Y-%m-%d %H:%M:%S")
        period_list.append(time)
        week_count = 'week' + '%s' % counter
        period_dict[week_count] = time
        
        time_counter += datetime.timedelta(days = 7)
        counter += 1
        
period_generator(start_time)
period_list
#period_dict

['2021-12-30 00:00:00',
 '2022-01-06 00:00:00',
 '2022-01-13 00:00:00',
 '2022-01-20 00:00:00',
 '2022-01-27 00:00:00',
 '2022-02-03 00:00:00',
 '2022-02-10 00:00:00',
 '2022-02-17 00:00:00',
 '2022-02-24 00:00:00',
 '2022-03-03 00:00:00',
 '2022-03-10 00:00:00',
 '2022-03-17 00:00:00',
 '2022-03-24 00:00:00',
 '2022-03-31 00:00:00',
 '2022-04-07 00:00:00',
 '2022-04-14 00:00:00',
 '2022-04-21 00:00:00',
 '2022-04-28 00:00:00',
 '2022-05-05 00:00:00',
 '2022-05-12 00:00:00',
 '2022-05-19 00:00:00',
 '2022-05-26 00:00:00',
 '2022-06-02 00:00:00',
 '2022-06-09 00:00:00',
 '2022-06-16 00:00:00',
 '2022-06-23 00:00:00',
 '2022-06-30 00:00:00',
 '2022-07-07 00:00:00',
 '2022-07-14 00:00:00',
 '2022-07-21 00:00:00',
 '2022-07-28 00:00:00',
 '2022-08-04 00:00:00',
 '2022-08-11 00:00:00',
 '2022-08-18 00:00:00',
 '2022-08-25 00:00:00',
 '2022-09-01 00:00:00',
 '2022-09-08 00:00:00',
 '2022-09-15 00:00:00',
 '2022-09-22 00:00:00',
 '2022-09-29 00:00:00',
 '2022-10-06 00:00:00',
 '2022-10-13 00:

In [ ]:
tmp = {'tx_hash':tx_hash, 'method':'90411a32', 'in_token_address':in_token_address, 'in_token_amount':in_token_amount, 'out_token_address':out_token_address, 'out_token_amount':out_token_amount, 'error_code':0}
df_transaction_result.loc[len(df_transaction_result)]=tmp
time.sleep(2)

In [116]:
def order_count_by_datetime(period_dict, order_sheet, chain = 'all'):
    order_count_by_datetime = pd.DataFrame(columns = ['week', 'chain', 'start_date', 'count'])
    period_list = []
    
    #pd.DataFrame(columns=['tx_hash', 'method', 'in_token_address', 'in_token_amount', 'out_token_address', 'out_token_amount', 'error_code'])
    
    for i in period_dict:
        period_list.append(period_dict[i])

    for week_day in period_dict:
        week_start = period_dict[week_day]
        next_week_day = 'week' + str(int(week_day[4:]) + 1)
        
        try:
            a = period_dict[next_week_day]
        except:
            continue
        
        week_end = period_dict[next_week_day]
        
        # Pandas 添加数据
        #df = df[(df['col'] < -0.25) | (df['col'] > 0.25)]
        
        if chain == 'all':
            order_sheet_count = order_sheet[(week_start < order_sheet['CreateTime']) & (order_sheet['CreateTime'] < week_end)].shape[0]
            tmp = {'week': week_day, 'chain': chain, 'start_date': week_start, 'count': order_sheet_count}
            order_count_by_datetime.loc[len(order_count_by_datetime)]=tmp
        
        else:
            order_sheet_count = order_sheet[(week_start < order_sheet['CreateTime']) & (order_sheet['CreateTime'] < week_end) & (order_sheet['Chain'] == chain)].shape[0]
            tmp = {'week': week_day, 'chain': chain, 'start_date': week_start, 'count': order_sheet_count}
            order_count_by_datetime.loc[len(order_count_by_datetime)]=tmp

    return order_count_by_datetime

#all_order_sheet[all_order_sheet['CreateTime'] < '2023-05-01 00:00:00']

In [118]:
#all_order_period_data = order_count_by_datetime(period_dict, all_order_sheet)
#eth_order_period_data = order_count_by_datetime(period_dict, all_order_sheet, 'eth')
#bsc_order_period_data
#arbitrum_order_period_data
filled_order_period_data = order_count_by_datetime(period_dict, filled_order_sheet)

chainlist = ['all', 'eth', 'bsc', 'arbitrum', 'polygon', 'avax', 'optimism']

data_list = []

for chain in chainlist:
    all_order_period_data = order_count_by_datetime(period_dict, all_order_sheet, chain)
    data_list.append(all_order_period_data)
    
    #new_users_count_by_datetime.to_csv('new_users_count_by_datetime_' + chain + '.csv')


In [124]:
writer = pd.ExcelWriter('all_order_period_data.xlsx')

for i in range(0, len(chainlist)):
    data_list[i].to_excel(writer, sheet_name = chainlist[i])

writer.close()

#for i in data_list:
#    i.to_excel(writer, sheet_name = )

In [65]:
#all_order_period_data.to_csv('all_order_period_data.csv')
#eth_order_period_data.to_csv('all_order_period_data_eth.csv')
filled_order_period_data.to_csv('filled_order_period_data.csv')

In [110]:
# 统计新用户数量

def user_count(period_dict, order_sheet, chain = 0):
    new_users_count_by_datetime = pd.DataFrame(columns = ['week', 'chain', 'start_date', 'new_users_count'])
    period_list = []
    user_list = []
    
    for i in period_dict:
        period_list.append(period_dict[i])
        
    for week_day in period_dict:
        week_start = period_dict[week_day]
        next_week_day = 'week' + str(int(week_day[4:]) + 1)

        try:
            a = period_dict[next_week_day]
        except:
            continue

        week_end = period_dict[next_week_day]
    
        if chain == 0:
                # 计算新用户数量
                users = order_sheet[(week_start < order_sheet['CreateTime']) & (order_sheet['CreateTime'] < week_end)]['OrderMaker']

                prev_user_count = len(user_list)

                for user in users:
                    if user not in user_list:
                        user_list.append(user)

                tmp = {'week': week_day, 'chain': chain, 'start_date': week_start, 'new_users_count': len(user_list) - prev_user_count}
                new_users_count_by_datetime.loc[len(new_users_count_by_datetime)]=tmp
        else:
                users = order_sheet[(week_start < order_sheet['CreateTime']) & (order_sheet['CreateTime'] < week_end) & (order_sheet['Chain']==chain)]['OrderMaker']

                prev_user_count = len(user_list)

                for user in users:
                    if user not in user_list:
                        user_list.append(user)

                tmp = {'week': week_day, 'chain': chain, 'start_date': week_start, 'new_users_count': len(user_list) - prev_user_count}
                new_users_count_by_datetime.loc[len(new_users_count_by_datetime)]=tmp

    
    
    return new_users_count_by_datetime

In [97]:
new_users_count_by_datetime = user_count(period_dict, all_order_period_data, chain = 0)

In [112]:
#new_users_count_by_datetime
#new_users_count_by_datetime.to_csv('new_users_count_by_datetime.csv')

chainlist = ['eth', 'bsc', 'arbitrum', 'polygon', 'avax', 'optimism']

for chain in chainlist:
    new_users_count_by_datetime = user_count(period_dict, all_order_sheet, chain)
    new_users_count_by_datetime.to_csv('new_users_count_by_datetime_' + chain + '.csv')
    


In [114]:
user_count(period_dict, all_order_sheet, 'polygon').to_csv('new_users_count_by_datetime_polygon.csv')

In [68]:
now = datetime.date.today()
now

datetime.date(2023, 5, 31)

In [42]:
filled_order_sheet['Chain'].value_counts()

Chain
bsc          324
arbitrum     254
polygon      220
fantom       165
avax          72
optimism      55
harmony       23
eth           22
cronos         4
xdai           4
heco           2
okex           2
moonriver      2
Name: count, dtype: int64